In [ ]:
#
import os
import sys
import warnings
import dataclasses
from copy import deepcopy
from pathlib import Path
from typing import Dict, Union, List, Callable

#
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

# Stable Baseline3
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

#
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# from pyfolio import create_returns_tear_sheet
# from pyfolio import utils

#
from rl.plot.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from common.utils import now_time
from configuration.settings import ProjectDir, ExperimentDir
from rl.data.CompanyInfo import CompanyInfo
from rl.envs.StockTradingEnv import StockTradingEnv
from rl.experiments._1_same_bigger_data_fundamental.train import (
    Program,
    CustomDRLAgent,
    get_dataset,
    get_env_kwargs,
    dataset_name,
    base_cols,
    data_cols,
    ratios_cols,
    algorithm_name,
)

In [ ]:
def ignore_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)


ignore_warnings()

In [ ]:
def main():
    program = Program(
        prj_dir=ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing")),
        exp_dir=ExperimentDir(Path(os.getcwd())),
        DEBUG=False,
    )
    program.dataset = get_dataset(
        pd.read_csv(program.exp_dir.out.datasets.joinpath(f"{dataset_name}.csv"), index_col=0), purpose="test"
    )
    program.exp_dir.check_and_create_dirs()
    return program


program = main()

In [ ]:
print(base_cols)
print(data_cols)
print(ratios_cols)

In [ ]:
@dataclasses.dataclass
class LearnedAlgorithm:
    algorithm: str
    filename: Path
    learned_algorithm: Union[A2C, PPO, DDPG, A2C, TD3]
    df_account_value = pd.DataFrame()
    df_actions = pd.DataFrame()
    perf_stats_all = pd.DataFrame()


def get_algorithm(filename: Path):
    if "a2c" in filename.as_posix():
        return LearnedAlgorithm(algorithm="a2c", filename=filename, learned_algorithm=A2C.load(filename))


learned_algorithms = [get_algorithm(filepath) for filepath in program.exp_dir.out.algorithms.glob("*")]

In [ ]:
# Test Learned Algorithms

print(len(learned_algorithms))

for algo in learned_algorithms:
    env_kwargs = get_env_kwargs(program.dataset)
    env_gym = StockTradingEnv(df=program.dataset, **env_kwargs)
    algo.df_account_value, algo.df_actions = CustomDRLAgent.DRL_prediction(
        model=algo.learned_algorithm, environment=env_gym
    )

In [ ]:
# print(learned_algorithms[0].df_actions)
# print(learned_algorithms[0].df_account_value)

In [ ]:
metric = "Annual return"


def calc_performance_statistics(algos: List[LearnedAlgorithm]):
    for algo in algos:
        perf_stats_all = backtest_stats(account_value=algo.df_account_value)
        algo.perf_stats_all = pd.DataFrame(perf_stats_all)


calc_performance_statistics(learned_algorithms)

In [ ]:
def print_learned_algo(algos: List[LearnedAlgorithm]):
    for i in algos:
        try:
            print(i.perf_stats_all.loc[metric][0])
        except Exception as e:
            print(f"{e}")


print_learned_algo(learned_algorithms)

In [ ]:
sorted_list = sorted(learned_algorithms, key=lambda x: x.perf_stats_all.loc[metric][0])

In [54]:
# print_learned_algo(sorted_list)
# print(f"{sorted_list[len(sorted_list)].filename}")

best_idx = len(sorted_list) - 1
worst_idk = 0


def get_annual_return(algo: LearnedAlgorithm):
    return algo.perf_stats_all.loc[metric][0]


print(f"{get_annual_return(sorted_list[best_idx])}:  {sorted_list[best_idx].filename.name}")
print(f"{get_annual_return(sorted_list[worst_idk])}:  {sorted_list[worst_idk].filename.name}")

0.08263260068750422:  a2c_2022-12-20T13-16-18_50000.zip
-0.0534658598936103:  a2c_2022-12-20T13-16-18_90000.zip


In [ ]:
start = program.dataset["date"].min()
end = program.dataset["date"].max()
print(f"Start: {start}  -  End: {end}")

In [ ]:
# Baseline
baseline_df = get_baseline("^DJI", start, end)

In [ ]:
baseline_df

In [ ]:
print(baseline_df.index.min())
print(baseline_df.index.max())

In [ ]:
stats = backtest_stats(baseline_df, value_col_name="close")

In [ ]:
# account_value = df_account_value
# baseline_ticker = "^DJI"
# baseline_start = program.dataset.index.min()
# baseline_end = program.dataset.index.max()
# value_col_name = "account_value"
#
# df = deepcopy(account_value)
# df["date"] = pd.to_datetime(df["date"])
# test_returns = get_daily_return(df, value_col_name=value_col_name)
#
# baseline_df = get_baseline(ticker=baseline_ticker, start=baseline_start, end=baseline_end)
#
# baseline_df["date"] = pd.to_datetime(baseline_df["date"], format="%Y-%m-%d")
# baseline_df = pd.merge(df[["date"]], baseline_df, how="left", on="date")
# baseline_df = baseline_df.fillna(method="ffill").fillna(method="bfill")
# baseline_returns = get_daily_return(baseline_df, value_col_name="close")
#
# with pyfolio.plotting.plotting_context(font_scale=1.1):
#     pyfolio.create_full_tear_sheet(
#         returns=test_returns,
#         benchmark_rets=baseline_returns,
#         set_context=False,
#     )
#

In [ ]:
#
# df = deepcopy(account_value)
# df["date"] = pd.to_datetime(df["date"])
# test_returns = get_daily_return(df, value_col_name=value_col_name)
#
# positions = utils.check_intraday("infer", test_returns, None, None)
#
# create_returns_tear_sheet(
#     test_returns,
#     positions=positions,
#     transactions=None,
#     live_start_date=None,
#     cone_std=(1.0, 1.5, 2.0),
#     benchmark_rets=None,
#     bootstrap=False,
#     turnover_denom="AGB",
#     header_rows=None,
#     set_context=True
# )